In [ ]:
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig
import transformers
import torch

In [ ]:
MAX_LEN = 200
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCH = 1
LEARNING_RATE= 1e-05
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass,self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768,10)
        
    def forward(self, ids, mask, token_type_ids):
        _, output1 = self.l1(ids, attention_mask=mask, token_type_ids=token_type_ids)
        output2 = self.l2(output1)
        output = self.l3(output2)
        return output
    
model  = BERTClass()
model.to(device='cpu')

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr = LEARNING_RATE)
def loss_fn(outputs,targets):
    loss_fn = torch.nn.BCEWithLogitsLoss()(outputs,targets)

In [ ]:
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader,0):
        ids = data['ids'].to(device, dtype=torch.long)
        mask = data['mask'].to(device, dtype=torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
        targets = data['targets'].to(device,dtype=torch.long)
        
        outputs = model(ids, mask, token_type_ids)
        
        loss = loss_fn(outputs,targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
for epoch in range(EPOCHS):
    train(epoch)

In [ ]:
def validataion(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _,data in enumerate(testing_loader,0):
            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            target = data['target'].to(device,dtype=torch.float)
            output = model(ids,mask,token_type_ids)
            
            fin_targets.extend(target.cpu().detach().numpy().tolist())  # torch -> numpy ->list
            fin_outputs.extend(torch.sigmoid(output).cpu().detach().numpy().tolist())   # torch -> numpy -> list
    return fin_outputs, fin_targets